In [1]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer,AutoConfig
from transformers import DataCollatorForTokenClassification
import torch
import re
import ast

In [ ]:
from bson.objectid import ObjectId

In [ ]:
import transformers

In [ ]:
torch.__version__

In [2]:
AutoModelForTokenClassification

transformers.models.auto.modeling_auto.AutoModelForTokenClassification

In [3]:
df = pd.read_csv(r"C:\Downloads\Train.csv")
L = [x for x in list(df.columns) if "_" not in x]
label_list = []
for m in L:
    label_list.append("B-"+m)
    label_list.append("I-"+m)
label_list.append("O")
label_list

['B-MedicationForm',
 'I-MedicationForm',
 'B-Age',
 'I-Age',
 'B-Frequency',
 'I-Frequency',
 'B-MedicationRoute',
 'I-MedicationRoute',
 'B-MedicationClass',
 'I-MedicationClass',
 'B-MeasurementUnit',
 'I-MeasurementUnit',
 'B-GeneOrProtein',
 'I-GeneOrProtein',
 'B-Course',
 'I-Course',
 'B-Gender',
 'I-Gender',
 'B-Dosage',
 'I-Dosage',
 'B-ConditionScale',
 'I-ConditionScale',
 'B-MedicationName',
 'I-MedicationName',
 'B-MeasurementValue',
 'I-MeasurementValue',
 'B-Allergen',
 'I-Allergen',
 'B-RelationalOperator',
 'I-RelationalOperator',
 'B-Diagnosis',
 'I-Diagnosis',
 'B-BodyStructure',
 'I-BodyStructure',
 'B-ConditionQualifier',
 'I-ConditionQualifier',
 'B-CareEnvironment',
 'I-CareEnvironment',
 'B-AdministrativeEvent',
 'I-AdministrativeEvent',
 'B-Time',
 'I-Time',
 'B-FamilyRelation',
 'I-FamilyRelation',
 'B-MutationType',
 'I-MutationType',
 'B-ExaminationName',
 'I-ExaminationName',
 'B-Variant',
 'I-Variant',
 'B-TreatmentName',
 'I-TreatmentName',
 'B-Direction'

In [4]:
L

['MedicationForm',
 'Age',
 'Frequency',
 'MedicationRoute',
 'MedicationClass',
 'MeasurementUnit',
 'GeneOrProtein',
 'Course',
 'Gender',
 'Dosage',
 'ConditionScale',
 'MedicationName',
 'MeasurementValue',
 'Allergen',
 'RelationalOperator',
 'Diagnosis',
 'BodyStructure',
 'ConditionQualifier',
 'CareEnvironment',
 'AdministrativeEvent',
 'Time',
 'FamilyRelation',
 'MutationType',
 'ExaminationName',
 'Variant',
 'TreatmentName',
 'Direction',
 'Expression',
 'SymptomOrSign',
 'HealthcareProfession']

In [5]:
label_encoding_dict = {}
for i in range(len(label_list)):
    label_encoding_dict[label_list[i]] = i
label_encoding_dict

{'B-MedicationForm': 0,
 'I-MedicationForm': 1,
 'B-Age': 2,
 'I-Age': 3,
 'B-Frequency': 4,
 'I-Frequency': 5,
 'B-MedicationRoute': 6,
 'I-MedicationRoute': 7,
 'B-MedicationClass': 8,
 'I-MedicationClass': 9,
 'B-MeasurementUnit': 10,
 'I-MeasurementUnit': 11,
 'B-GeneOrProtein': 12,
 'I-GeneOrProtein': 13,
 'B-Course': 14,
 'I-Course': 15,
 'B-Gender': 16,
 'I-Gender': 17,
 'B-Dosage': 18,
 'I-Dosage': 19,
 'B-ConditionScale': 20,
 'I-ConditionScale': 21,
 'B-MedicationName': 22,
 'I-MedicationName': 23,
 'B-MeasurementValue': 24,
 'I-MeasurementValue': 25,
 'B-Allergen': 26,
 'I-Allergen': 27,
 'B-RelationalOperator': 28,
 'I-RelationalOperator': 29,
 'B-Diagnosis': 30,
 'I-Diagnosis': 31,
 'B-BodyStructure': 32,
 'I-BodyStructure': 33,
 'B-ConditionQualifier': 34,
 'I-ConditionQualifier': 35,
 'B-CareEnvironment': 36,
 'I-CareEnvironment': 37,
 'B-AdministrativeEvent': 38,
 'I-AdministrativeEvent': 39,
 'B-Time': 40,
 'I-Time': 41,
 'B-FamilyRelation': 42,
 'I-FamilyRelation': 43

In [ ]:
task = "ner" 
model_checkpoint = "d4data/biomedical-ner-all"
batch_size = 16

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
def Pre_process_clean(text):
    try :
        text = re.sub(r'\(','',text)
        text = re.sub(r"\)",'',text)
        text = re.sub(r"\.",'',text)
        text = re.sub(r"|",'',text)
        text = re.sub(r'[-#&@*/\',:;"]','',text)
        return text
    except:
        return text

In [ ]:
def List_Tokenizer(text):
    tokens = tokenizer(text)
    words = tokenizer.batch_decode(tokens['input_ids'])
    return words

def Tokenizer_Label(Label_words):
    L = []
    try :
        if Label_words[0] == 'n':
            return ''
        for x in Label_words:
            w = List_Tokenizer(x)
            for i in w:
                if (i != '[CLS]') and (i != '[SEP]'):
                    L.append(i)
    except :
        return ''
    return L

def Ner_tagging(T,MT):
    T['ner_tags'] = 0
    for i in range(T.shape[0]):
        print(f"Sent : {T['original_text'][i]}")
        NER = []
        L = T['tokens'][i]
        Master_Dic_words = {}
        Master_Dic_words_tag = {}
        JT = []
        for k in MT:
            j = 0
            for tp in str(T[k][i]).split("||"):
                JT.append(k+"_"+str(j))
                Master_Dic_words[k+"_"+str(j)+"_words"] = [t.lower() for t in str(tp).split()]
                Master_Dic_words_tag[k+"_"+str(j)+"_words_tag"] = Tokenizer_Label(Master_Dic_words[k+"_"+str(j)+"_words"])
                j = j + 1
        Master_Dic_Start = {}
        Master_Dic_Sec = {}
        for k in JT:
            Master_Dic_Start["Start_"+k] = Master_Dic_words_tag[k+"_words_tag"][0]
            try :
                Master_Dic_Sec["Sec_"+k] = Master_Dic_words_tag[k+"_words_tag"][1]
            except :
                Master_Dic_Sec["Sec_"+k] = ""
                
        prev_tag = ""
        k = 0
        for j in L:
            f = 0
            first = "Yes"
            for mt in JT:
                mtn = mt.split("_")[0].strip()
                if first == "Yes":
                    first = "No"
                    if j.lower() in Master_Dic_words_tag[mt+"_words_tag"]:
                        f = 1
                        if (prev_tag == "B-"+mtn) or (prev_tag == "I-"+mtn):
                            NER.append('I-'+mtn)
                            prev_tag = "I-"+mtn
                        else :
                            if j.lower() == Master_Dic_Start["Start_"+mt] and len(Master_Dic_words_tag[mt+"_words_tag"]) == 1:
                                NER.append('B-'+mtn)
                                prev_tag = "B-"+mtn
                                Start_Ent = "NAN"
                            elif j.lower() == Master_Dic_Start["Start_"+mt] and (L[k+1].lower() == Master_Dic_Sec["Sec_"+mt] or L[k+1].lower() == ""):
                                NER.append('B-'+mtn)
                                prev_tag = "B-"+mtn
                                Start_Ent = "NAN"
                            else :
                                f = 0
                else :
                    if j.lower() in Master_Dic_words_tag[mt+"_words_tag"] and f == 0:
                        f = 1
                        if (prev_tag == "B-"+mtn) or (prev_tag == "I-"+mtn):
                            NER.append('I-'+mtn)
                            prev_tag = "I-"+mtn
                        else :
                            if j.lower() == Master_Dic_Start["Start_"+mt]  and (L[k+1].lower() == Master_Dic_Sec["Sec_"+mt]  or L[k+1].lower() == ""):
                                NER.append('B-'+mtn)
                                prev_tag = "B-"+mtn
                                Start_HOP = "NAN"
                            else :
                                f = 0

            if f == 0 :
                NER.append('O')
                prev_tag = "O"
            k = k + 1
        T['ner_tags'][i] = str(NER)
    return T

In [ ]:
T = pd.read_csv(r"C:\Downloads\Train.csv")
T.replace('0','NAN', inplace=True)
T.replace('','NAN', inplace=True)
T.head()

In [ ]:
T.head()

In [ ]:
T['tokens'] = T['original_text'].apply(lambda x : List_Tokenizer(str(x)))
T = Ner_tagging(T,L)

In [ ]:
T.head()

In [ ]:
ti = T[T['ner_tags']==0]

In [ ]:
ti.shape

In [ ]:
T.shape

In [ ]:
T['original_text'][8]

In [ ]:
T['tokens'][8]

In [ ]:
T['ner_tags'][8]

In [ ]:
Train = T[['ner_tags','tokens']]
Train.head()
Train.shape

In [ ]:
def get_un_token_dataset(train_df, test_df):
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)
    return (train_dataset, test_dataset)

In [ ]:
Train = Train.sample(frac=1, axis=1).sample(frac=1).reset_index(drop=True)
Train.head()

In [ ]:
Train['ner_tags'] = Train['ner_tags'].apply(lambda x : ast.literal_eval(str(x)))
Train['tokens'] = Train['tokens'].apply(lambda x : ast.literal_eval(str(x)))

In [ ]:
for row in range(Train.shape[0]):
    if len(Train['ner_tags'][row]) == len(Train['tokens'][row]):
        continue
    else :
        print(f"Error in row number : {row}")

In [ ]:
Train.shape

In [ ]:
train_df = Train.iloc[:10000,:]
test_df = Train.iloc[10000:,:]
train_dataset, test_dataset = get_un_token_dataset(train_df, test_df)

In [ ]:
train_dataset.data

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_tokenized = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized = test_dataset.map(tokenize_and_align_labels, batched=True)

In [ ]:
train_tokenized

In [ ]:
"""
This Method uses the config but uses random weights instead of pretrained weights of BioBert-NER
"""
config = AutoConfig.from_pretrained(model_checkpoint)

In [ ]:
"""
This Method removes top Classification layer and uses pretrained weights of BioBert-NER 
"""
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
model.classifier = torch.nn.Linear(in_features=model.classifier.in_features, out_features=config.num_labels, bias=True)
model.config = config    
model.num_labels = config.num_labels

In [ ]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=13,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.evaluate()
trainer.save_model('BIONER-MODEL-2')

In [6]:
import re
def clean(text):
    try :
        text = re.sub(r'[-#&@*/{}():;.,]','',text)
        return text
    except Exception as ex:
        print(ex)
        return text

In [7]:
def get_scispacy_tags(text):

        text = clean(text)
        T = []
        df = pd.read_csv(r"C:\Downloads\BIONER-DOCKER\Medication.csv")
        L = list(df['Medication'])
        for j in L:
            j = clean(j)
            if j.strip().upper() in text.upper() and len(j) > 3 and len(j.strip()) > 0:
                j_words = j.strip().upper().split()
                text_words = text.upper().split()
                f = 1
                for wd in j_words:
                    if wd not in text_words:
                        f = 0
                        break
                if f == 1:
                    T.append(["MedicationName",j.strip(),len(j.strip())])
        df = pd.read_csv(r"C:\Downloads\BIONER-DOCKER\Diagnosis.csv")
        L = list(df['Diagnosis'])
        for j in L:
            j = clean(j)
            if j.strip().upper() in text.upper() and len(j) > 3 and len(j.strip()) > 0:
                j_words = j.strip().upper().split()
                text_words = text.upper().split()
                f = 1
                for wd in j_words:
                    if wd not in text_words:
                        f = 0
                        break
                if f == 1:
                    T.append(["Diagnosis",j.strip(),len(j.strip())])
        M = sorted(T,key=lambda x : x[2],reverse=True)
        Z = [x[:2] for x in M]
        return Z

In [21]:
D = pd.read_csv(r"C:\Downloads\training.csv")

In [23]:
tokenizer = AutoTokenizer.from_pretrained('./BIONER-MODEL/')

In [24]:
L1_sentences = []
Preds = []
Texts = []
model = AutoModelForTokenClassification.from_pretrained('./BIONER-MODEL/')
for paragraph in D['original_text']:
    Texts.append(paragraph)
#     paragraph = Pre_process_clean(paragraph)
    tokens = tokenizer(paragraph)
    torch.tensor(tokens['input_ids']).unsqueeze(0).size()

    predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
    predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
    predictions = [label_list[i] for i in predictions]

    words = tokenizer.batch_decode(tokens['input_ids'])
    L1_sentences.append(words)
    Preds.append(predictions)
E = pd.DataFrame(list(zip(L1_sentences,Preds)),columns=["L1_Sents","Preds"])

In [28]:
for mt in L:
    C[mt] = 0

In [29]:
def entity_identification(words,predictions,tag1,tag2):
    NER =[]
    NER_tag = []
    Pred_Lab = []
    Pred_tag = []
    prev_tag = ''
    ent = ''
    tag = ''
    start_ind = 0
    end_ind = 0
#     print(f"Words : {words}")
#     print(f"Predictions : {predictions}")
    for i in range(len(predictions)-1):
        if (predictions[i] == tag1 or predictions[i] == tag2) and (tag != tag1):
            start_ind = i
#             print(f"start_ind : {start_ind}")
            tag = tag1
        if ((predictions[i] == tag2 or predictions[i] == tag1 or '##' in words[i]) and (predictions[i+1] != tag2 and ('##' not in words[i+1]))  and (tag == tag1)):
            end_ind = i
#             print(f"end_ind : {end_ind}")
            tag = tag2
        if (tag == tag2) and (end_ind != 0):
            while ('##' in words[start_ind]):
                start_ind = start_ind - 1
            for j in range(start_ind,end_ind+1):
#                 if words[j] not in NER:
                NER.append(words[j])
                if j == start_ind:
                    Pred_tag.append(tag1)
                else :
                    Pred_tag.append(predictions[j])
#                     Pred_tag[words[j]] = predictions[j]

            start_ind = 0
            end_ind = 0
            tag = ''
#         if (predictions[i] == 'B-RP') or (predictions[i] == 'I-RP'):
#             NER.append(words[i])
#     print(f"NER : {NER}")
#     print(f"Pred_tag : {Pred_tag}")
    if len(NER) != 0:
        k = 0
        for j in NER:
            if ('##' in j) or (Pred_tag[k] != tag1):
                if ('##' in j):
                    ent = ent + j.replace('##', '')
                else :
                    ent = ent + " " + j
                prev_tag = '##'
            else :
                if prev_tag == '##' or prev_tag == 'N#':
                    NER_tag.append(ent)
                    Ent = ent
                    if (Ent not in Pred_Lab and Ent != ""):
                        Pred_Lab.append(Ent)
                ent = j
                prev_tag = 'N#'
            k = k + 1
        NER_tag.append(ent)
        Ent = ent
        Pred_Lab_words = [sub.split() for sub in Pred_Lab]
        if (Ent not in Pred_Lab and Ent not in Pred_Lab_words and Ent != ""):
            Pred_Lab.append(Ent)
        Pred_Lab_clean = []
        for e in range(len(Pred_Lab)):
            f = 0
            for k in range(e+1,len(Pred_Lab)):
                if Pred_Lab[e] in Pred_Lab[k].split():
                    f = 1
            if f == 0:
                Pred_Lab_clean.append(Pred_Lab[e])
#         print(f"Pred_Lab : {Pred_Lab_clean}")
        A = ",".join([x for x in Pred_Lab_clean])
    else :
        A = ""
    return str(A)

In [30]:
import ast
words = list(E['L1_Sents'])
predictions = list(E['Preds'])

In [2]:
for i in range(len(words)):
    for mt in L:
        A = entity_identification(words[i],predictions[i],'B-'+mt,'I-'+mt)
        if mt == "Diagnosis" or mt == "MedicationName":
            text = Texts[i]
            T = get_scispacy_tags(text)
            B = ""
            for G in T:
                if G[0] == mt:
                    B = B + "," + G[1]
            A = B + "," + A
        C[mt][i] = A
#         Master_Entities[mt].append(A)
#         print("------------------------------------\n")